In [1]:
import os
os.environ["TMPDIR"] = "/tmp"  # set the folder for temporary files

from qiskit.opflow.gradients import Gradient, NaturalGradient
from qiskit_mod.qiskit_ter import LinCombFullmod,LinCombMod
from qiskit_nature.algorithms import VQEUCCFactory,GroundStateEigensolver
from qiskit.algorithms.optimizers import SLSQP,L_BFGS_B,SPSA, COBYLA
from qiskit_nature.circuit.library import HartreeFock
from qiskit.circuit.library import TwoLocal
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.drivers.second_quantization.pyscfd import PySCFDriver
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
from qiskit import Aer
from qiskit.utils import QuantumInstance

/home/gsilvi/miniforge3/envs/NEASQC4wMYQLM/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
molecule = 'H .0 {0} {0}; H .0 {0} -{0};'
orbitals_to_remove = []
basis = 'sto3g'
optimizer = L_BFGS_B(maxiter=1000)
qubit_converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True) 
molecule_set = molecule.format(1/2)
driver = PySCFDriver(atom=molecule_set,
                        unit=UnitsType.ANGSTROM,
                        basis=basis)
#qmolecule = driver.run()
es_problem = ElectronicStructureProblem(driver,
                                        q_molecule_transformers = [FreezeCoreTransformer(freeze_core = True,
                                                                                        remove_orbitals = orbitals_to_remove
                                                                                        )]
                                        )
second_q_op = es_problem.second_q_ops()
qubit_op = qubit_converter.convert(second_q_op[0],
                                    num_particles = es_problem.num_particles) #this line is useful for initialize the converter and get the right hartree-fock number of qubits
particle_number = es_problem.properties_transformed.get_property("ParticleNumber")

In [3]:
from random import seed


shots = 1024
qiskitbackend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend=qiskitbackend,seed_simulator=42, seed_transpiler=42, shots=10000)
from qat.qpus import PyLinalg
qlm_qpu = PyLinalg()

In [4]:
from qiskit.circuit import ParameterExpression, ParameterVector, Parameter
from typing import Callable, Iterable, List, Optional, Tuple, Union,  cast ,Dict
from qiskit.providers import BaseBackend, Backend
from qiskit.opflow import ExpectationBase, CircuitSampler, PauliExpectation, OperatorStateFn, CircuitStateFn, PauliOp, ListOp, SummedOp,ComposedOp
from qiskit.quantum_info import Pauli
from qiskit_mod.wrapper2myqlm import *
LIST_OF_GATES = ['i', 'id', 'iden', 'u', 'u0', 'u1', 'u2', 'u3', 'x', 'y', 'z', 'h', 's', 'sdg', 't', 'tdg', 'rx', 'ry', 'rz', 'cx', 'cy', 'cz', 'ch', 'crz', 'cu1', 'cu3', 'swap', 'ccx', 'cswap', 'r']

def extract_and_compute_circ(self, operator: OperatorBase,prev_coeff=1) -> None:
            """
            Recursively extract the ``CircuitStateFns`` contained in operator into the
            ``_circuit_ops_cache`` field.
            """
            # print(operator,'vs',operator.coeff)
            #print('Coeff',operator.coeff, type(operator))
            if isinstance(operator, ComposedOp):
               
                coefficient = prev_coeff*operator.coeff
                for i,op in enumerate(operator.oplist):
                    if isinstance(op,CircuitStateFn):
                        circuit = op.to_circuit_op()
                        #coefficient*=op.coeff
                    elif isinstance(op,OperatorStateFn):
                        operator_to_run = op
                        #coefficient*=op.coeff
                    elif isinstance(op,ListOp):
                        for op2 in op.oplist:
                            if isinstance(op2,CircuitStateFn):
                                circuit = op2.to_circuit_op()
                                #coefficient*=op2.coeff
                if circuit is not None and operator_to_run is not None:
                    value = run_circuit_in_QLM(self,circuit,operator_to_run, coefficient)
                    #print('Computed',value, 'with coefficient',coefficient)
                return value
                
            elif isinstance(operator, SummedOp):
                list_of_values=[]
                coefficient = prev_coeff*operator.coeff
                for i,op in enumerate(operator.oplist):
                    list_of_values.append(extract_and_compute_circ(self, op,prev_coeff=coefficient))
                summation= sum(list_of_values)
                #print('Computed sum',summation)
                return summation

            elif isinstance(operator, ListOp):
                list_of_values=[]
                
                coefficient = prev_coeff*operator.coeff
                #print(coefficient)
                for i,op in enumerate(operator.oplist):
                    list_of_values.append(extract_and_compute_circ(self, op, prev_coeff=coefficient))
                return list_of_values



def run_circuit_in_QLM(self,circuit,operator, coefficient):

    operator_pauli = operator._primitive.to_pauli_op()
    operator_meas_inv = PauliOp(primitive= Pauli(operator_pauli.primitive.to_label()[::-1]), coeff=1)#operator_pauli.coeff)
    transpiled_circ = transpile(circuit.to_circuit(),
                            basis_gates=LIST_OF_GATES,
                            optimization_level=0)
    qcirc = qiskit_to_qlm(transpiled_circ)
    job = qcirc.to_job(observable=Observable(operator_meas_inv.num_qubits, matrix=operator_meas_inv.to_matrix()),nbshots=self.nb_shots) 
    result_temp = self.execute(job)
    return coefficient * np.real(result_temp.value)




def gradient_wrapper_for_QLM(
        self,
        operator: OperatorBase,
        bind_params: Union[ParameterExpression, ParameterVector, List[ParameterExpression]],
        grad_params: Optional[
            Union[
                ParameterExpression,
                ParameterVector,
                List[ParameterExpression],
                Tuple[ParameterExpression, ParameterExpression],
                List[Tuple[ParameterExpression, ParameterExpression]],
            ]
        ] = None,
        backend: Optional[Union[BaseBackend, Backend, QuantumInstance]] = None,
        expectation: Optional[ExpectationBase] = None,
    ) -> Callable[[Iterable], np.ndarray]:
        """Get a callable function which provides the respective gradient, Hessian or QFI for given
        parameter values. This callable can be used as gradient function for optimizers.

        Args:
            operator: The operator for which we want to get the gradient, Hessian or QFI.
            bind_params: The operator parameters to which the parameter values are assigned.
            grad_params: The parameters with respect to which we are taking the gradient, Hessian
                or QFI. If grad_params = None, then grad_params = bind_params
            backend: The quantum backend or QuantumInstance to use to evaluate the gradient,
                Hessian or QFI.
            expectation: The expectation converter to be used. If none is set then
                `PauliExpectation()` is used.

        Returns:
            Function to compute a gradient, Hessian or QFI. The function
            takes an iterable as argument which holds the parameter values.
        """
        from qiskit.opflow.converters import CircuitSampler


        if not grad_params:
            grad_params = bind_params

        grad = self.convert(operator, grad_params)
        if expectation is None:
            expectation = PauliExpectation()
        grad = expectation.convert(grad)
        #print(grad)
        #print('General operator',operator)
        #print('bind_params',bind_params)


        def gradient_fn(p_values):
            p_values_dict = dict(zip(bind_params, p_values))
            if not backend:
                converter = grad.assign_parameters(p_values_dict)
                return np.real(converter.eval())
            else:
                p_values_dict = {k: [v] for k, v in p_values_dict.items()} # remake the dict with list of values
                # converter = grad.assign_parameters(p_values_dict) # assign the values to the parameters
                #print(converter)
                #converted_circ = expectation.convert(grad)
                circuit_assigned = grad.assign_parameters(p_values_dict) # assign the values to the parameters
                dictio = extract_and_compute_circ(self,circuit_assigned) 
                #print('LIST',dictio[0], dictio[1])
                
                # set up the matrix here
                val = dictio[1]
                matrix = np.zeros((len(val),len(val)))
                upper_w_diag = np.triu_indices(len(val))
                matrix[upper_w_diag] = np.array(np.concatenate(val))
                i_upper = np.triu_indices(len(val), 1)
                i_lower = np.tril_indices(len(val), -1)
                matrix[i_lower] = matrix[i_upper]

                
                converter = CircuitSampler(backend=backend).convert(grad, p_values_dict)
                #print('Qiskit first 5 circuits',[op.eval() for op in converter[0][0][0].oplist])
                #print('Qiskit first sums',[op.eval() for op in converter[0][0].oplist])
                QLM_result = converter[0].combo_fn([dictio[0],matrix])
                # print('QLM_result',QLM_result)
                # print('Qiskit result',np.real(converter.eval()[0]))

                return QLM_result
                

        return gradient_fn

In [5]:
def create_and_run_job(self,
                       operator_meas: OperatorBase,
                       params: dict):
            """ Compose the qlm job from ansatz binded to params and measure operato on it"""
            # check if the parameters passed are as a range or single value
            if params is not None and len(params.keys()) > 0:
                p_0 = list(params.values())[0]
                if isinstance(p_0, (list, np.ndarray)):
                    num_parameterizations = len(p_0)
                    param_bindings = [
                        {param: value_list[i] for param, value_list in params.items()}
                        for i in range(num_parameterizations)
                    ]
                else:
                    num_parameterizations = 1
                    param_bindings = [params]

            else:
                param_bindings = None
                num_parameterizations = 1
            results = []
            
            for circ_params in param_bindings:
                ansatz_in_use = self._ansatz.assign_parameters(circ_params)
                transpiled_circ = transpile(ansatz_in_use,
                                            basis_gates=LIST_OF_GATES,
                                            optimization_level=0)
                qcirc = qiskit_to_qlm(transpiled_circ)
                job = qcirc.to_job(observable=Observable(operator_meas.num_qubits, matrix=operator_meas.to_matrix()),nbshots=self.nb_shots) 
                # print('Shots= ',job.nbshots)
                # START COMPUTATION
                result_temp = self.execute(job)

                if isinstance(result_temp, AsyncResult):  # chek if we are dealing with remote job
                    #case for local plugin/remote qpu
                    result = result_temp.join()
                    
                else:
                    result = result_temp
                #Qiskit check
                # sampler = CircuitSampler(self.quantum_instance)
                # exp = ~StateFn(operator_meas) @ CircuitStateFn(transpiled_circ)
                # converted = sampler.convert(exp)
                # print('Evaluation (qiskit)',converted.eval(),'vs QLM',result.value)

                results.append(result.value)
            
            return results

def get_energy_evaluation_QLM(
        self,
        operator: OperatorBase,
        return_expectation: bool = False,
    ) -> Callable[[np.ndarray], Union[float, List[float]]]:
        """Returns a function handle to evaluates the energy at given parameters for the ansatz.

        This is the objective function to be passed to the optimizer that is used for evaluation.

        Args:
            operator: The operator whose energy to evaluate.
            return_expectation: If True, return the ``ExpectationBase`` expectation converter used
                in the construction of the expectation value. Useful e.g. to evaluate other
                operators with the same expectation value converter.
            MODIFIED: submit the job to QLM qpus


        Returns:
            Energy of the hamiltonian of each parameter, and, optionally, the expectation
            converter.

        Raises:
            RuntimeError: If the circuit is not parameterized (i.e. has 0 free parameters).

        """
        num_parameters = self.ansatz.num_parameters
        if num_parameters == 0:
            raise RuntimeError("The ansatz must be parameterized, but has 0 free parameters.")

        expect_op, expectation = self.construct_expectation(
            self._ansatz_params, operator, return_expectation=True
        )
        reversed_operator = sum([PauliOp(primitive= Pauli(op.to_pauli_op().primitive.to_label()[::-1]), coeff=op.to_pauli_op().coeff) for op in operator])
        #print('Operator type',type(reversed_operator), reversed_operator)
        #print('Here',expectation.convert(StateFn(operator, is_measurement=True)).to_matrix_op())

        

        def energy_evaluation(parameters):
            parameter_sets = np.reshape(parameters, (-1, num_parameters))
            param_bindings = dict(zip(self._ansatz_params, parameter_sets.transpose().tolist())) # combine values to parameters symbols
            #print('param_bindings',param_bindings)
            sampled_expect_op = self._circuit_sampler.convert(expect_op, params=param_bindings)
            print('Qiskit result EE = ',np.real(sampled_expect_op.eval()))
            means = np.real(create_and_run_job(self,reversed_operator, param_bindings)) #important to reverse the operator because of different conventions QLM/Qiskit
            print('QLM result EE  =',means)
            if self._callback is not None:
                parameter_sets = np.reshape(parameters, (-1, num_parameters))
                param_bindings = dict(zip(self._ansatz_params, parameter_sets.transpose().tolist()))
                variance = np.real(expectation.compute_variance(self._circuit_sampler.convert(expect_op, params=param_bindings)))
                estimator_error = np.sqrt(variance / self.quantum_instance.run_config.shots)
                for i, param_set in enumerate(parameter_sets):
                    self._eval_count += 1
                    self._callback(self._eval_count, param_set, means[i], estimator_error[i])
            else:
                self._eval_count += len(means)
            return means if len(means) > 1 else means[0]

        if return_expectation:
            return energy_evaluation, expectation

        return energy_evaluation

In [6]:
from qiskit_mod.wrapper2myqlm import *
from qiskit_mod.my_junction import IterativeExplorationVQE,get_energy_evaluation_QLM
from qiskit.opflow.gradients import DerivativeBase

#
use_QLM = True
if use_QLM:
    VQE.get_energy_evaluation = get_energy_evaluation_QLM #override the function, class-wide
    DerivativeBase.gradient_wrapper = gradient_wrapper_for_QLM
    plugin_in_use = IterativeExplorationVQE

In [7]:
nat_grad = NaturalGradient(grad_method=LinCombMod(img=False),
                               qfi_method=LinCombFullmod(),
                               regularization='ridge')
hartree_fock_state = HartreeFock(particle_number.num_spin_orbitals,
                                    (particle_number.num_alpha, particle_number.num_beta),
                                    qubit_converter)
num_qubits = hartree_fock_state.num_qubits
ansz = TwoLocal(hartree_fock_state.num_qubits,
                    rotation_blocks=['ry'], 
                    entanglement_blocks='cx',
                    entanglement="full",
                    reps=1,
                    initial_state=hartree_fock_state)
Im_solver = VQEUCCFactory(quantum_instance,
                            gradient=nat_grad,
                            optimizer=L_BFGS_B(maxiter=1000), # good optimizer for Nat. Grad.
                            ansatz=ansz.decompose())



In [8]:
calcIevo = GroundStateEigensolver(qubit_converter,
                                    Im_solver)

if use_QLM:
    stack = build_QLM_stack(calcIevo,molecule_set,
                        plugin_in_use,
                        qlm_qpu,
                        shots = 0,
                        remove_orbitals = orbitals_to_remove)
    resIevo = run_QLM_stack(stack,num_qubits,0)
    # resIevo = calcIevo.solve(es_problem)

    print('Imaginary time evo. ,',
            resIevo.total_energies, ',exec_time=',
            len(resIevo.raw_result.optimal_parameters))
else:
    resIevo= calcIevo.solve(es_problem)
    print('Imaginary time evo. ,',
        resIevo.total_energies, ',exec_time=',
        len(resIevo.raw_result.optimal_parameters))

0
Imaginary time evo. , -1.1011502603283905 ,exec_time= 4


/home/gsilvi/.local/lib/python3.9/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


Good value to compare: -1.10115

In [9]:
from qiskit.opflow import (StateFn, Zero, One, Plus, Minus, H,
                           DictStateFn, VectorStateFn, CircuitStateFn, OperatorStateFn)
from qiskit.opflow import I, X, Y, Z 


opera =((-1.0692434904119297-8.326672684688674e-17j) * I^I) #- (0.267528649942086 * Z^I)\
#   +( (0.2675286499420861-1.3877787807814457e-17j) * I^Z)\
#   + ((-0.009014930058166226-1.3877787807814457e-17j) * Z^Z)\
#   + ((0.19679058348547016+3.469446951953614e-18j) * X^X)


In [10]:
gradf = calcIevo.solver.gradient.gradient_wrapper(~StateFn(opera) @ StateFn(ansz.decompose()),list(ansz.parameters), backend = quantum_instance)

In [11]:
conv =gradf([0.324, 0.54, 1.2, 9.99])

In [12]:
mat = np.asarray([[0.9999999999999996, 0.0, 0.5141359916531129, 0.0][0.0,0.9999999999999994,0.0, -0.9479695613221306], [0.9999999999999996, 0.27306097260886564], [0.9999999999999997]])
#mat = np.asarray([ [0.9999999999999997,0,0,0],[0.9999999999999996, 0.27306097260886564,0,0],[0.9999999999999994, 0.0, -0.9479695613221306,0],[0.9999999999999996, 0.0, 0.5141359916531129, 0.0]])


<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
/tmp/ipykernel_58739/3874327922.py:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  mat = np.asarray([[0.9999999999999996, 0.0, 0.5141359916531129, 0.0][0.0,0.9999999999999994,0.0, -0.9479695613221306], [0.9999999999999996, 0.27306097260886564], [0.9999999999999997]])


TypeError: list indices must be integers or slices, not tuple

In [ ]:
upper_w_diag = np.triu_indices(4)
i_upper = np.triu_indices(4, 1)
i_lower = np.tril_indices(4, -1)
matrix = np.zeros((4,4))
valus = [0.9999999999999996, 0.0, 0.5141359916531129, 0.0, 0.9999999999999994, 0.0, -0.9479695613221306, 0.9999999999999996, 0.27306097260886564, 0.9999999999999997]

matrix[upper_w_diag] = valus
matrix[i_lower] = matrix[i_upper]

conv[0].combo_fn([[0.09879583, 0.0264148,  0.32459114, 0.07658635],1/4*matrix])

array([-0.27994812,  0.32068078,  1.32423912,  0.24095422])

In [ ]:

val = [[0.9999999999999996, 0.0, 0.5141359916531129, 0.0], [0.9999999999999994, 0.0, -0.9479695613221306], [0.9999999999999996, 0.27306097260886564], [0.9999999999999997]]


matrix = np.zeros((4,4))
matrix[upper_w_diag] = np.asarray(valus)
matrix

array([[ 1.        ,  0.        ,  0.51413599,  0.        ],
       [ 0.        ,  1.        ,  0.        , -0.94796956],
       [ 0.        ,  0.        ,  1.        ,  0.27306097],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [ ]:

matrix = np.zeros((len(val),len(val)))
upper_w_diag = np.triu_indices(len(val))
matrix[upper_w_diag] = np.array(np.concatenate(val))
i_upper = np.triu_indices(len(val), 1)
i_lower = np.tril_indices(len(val), -1)
matrix[i_lower] = matrix[i_upper]
print(matrix)

[[ 1.          0.          0.51413599  0.        ]
 [ 0.          1.          0.         -0.94796956]
 [ 0.51413599  0.          1.          0.27306097]
 [ 0.         -0.94796956  0.27306097  1.        ]]
